In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()


pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210706-1518-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [2]:
ht=hl.read_table(f'{lustre_dir}/variant_qc/models/91ba5f38/rf_result_MegaWES_new.ht')
ht_val_denovo=hl.read_table(f'{lustre_dir}/variant_qc/DDD_validated_denovo_b38.ht')

In [4]:
ht_val_denovo.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'ega': str 
    'chr': str 
    'pos': int32 
    'ref': str 
    'alt': str 
    'genotype': str 
    'inheritance': str 
    'mum': str 
    'dad': str 
    'in_mum': str 
    'in_dad': str 
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'reference_genome': str 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [6]:
ht_val_denovo.inheritance.summarize()

Non-missing,4926 (100.00%)
Missing,0
Min Size,7
Max Size,56
Mean Size,25.53
Sample Values,"['Unknown', 'Maternally inherited, constitutive in mother', 'Paternally inherited, constitutive in father', 'Paternally inherited, constitutive in father', 'De novo constitutive']"


In [7]:
ht_val_filtered=ht_val_denovo.filter(ht_val_denovo.inheritance=='De novo constitutive')

In [8]:
ht_val_denovo.count()

4926

In [9]:
ht_val_filtered.count()

2358

In [11]:
ht_val_filtered.write(f'{lustre_dir}/variant_qc/DDD_validated_denovo_b38_only_denovo_interitance.ht', overwrite=True)

2021-07-06 10:45:58 Hail: INFO: wrote table with 2358 rows in 100 partitions to file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/DDD_validated_denovo_b38_only_denovo_interitance.ht
    Total size: 120.89 KiB
    * Rows: 120.88 KiB
    * Globals: 11.00 B
    * Smallest partition: 2 rows (232.00 B)
    * Largest partition:  45 rows (2.28 KiB)


In [12]:
ht_val_filtered.show()

,,,,,,,,,,,,,
ega,chr,pos,ref,alt,genotype,inheritance,mum,dad,in_mum,in_dad,locus,alleles,reference_genome
str,str,int32,str,str,str,str,str,str,str,str,locus<GRCh38>,array<str>,str
"""EGAN00001405830""","""chr1""",1338107,"""CGGGTGGGGCAGCG""","""C""","""heterozygous""","""De novo constitutive""","""EGAN00001406454""","""EGAN00001406285""","""N""","""N""",chr1:1338107,"[""CGGGTGGGGCAGCG"",""C""]","""GRCh38"""
"""EGAN00001291289""","""chr1""",1349008,"""T""","""C""","""heterozygous""","""De novo constitutive""","""EGAN00001291287""","""EGAN00001291288""","""N""","""N""",chr1:1349008,"[""T"",""C""]","""GRCh38"""
"""EGAN00001292956""","""chr1""",1520282,"""G""","""A""","""heterozygous""","""De novo constitutive""","""EGAN00001292954""","""EGAN00001292955""","""N""","""N""",chr1:1520282,"[""G"",""A""]","""GRCh38"""
"""EGAN00001351798""","""chr1""",1527719,"""C""","""G""","""heterozygous""","""De novo constitutive""","""EGAN00001351800""","""EGAN00001351799""","""N""","""N""",chr1:1527719,"[""C"",""G""]","""GRCh38"""
"""EGAN00001300662""","""chr1""",1529299,"""C""","""T""","""heterozygous""","""De novo constitutive""","""EGAN00001300660""","""EGAN00001300661""","""N""","""N""",chr1:1529299,"[""C"",""T""]","""GRCh38"""
"""EGAN00001352523""","""chr1""",1790531,"""A""","""T""","""heterozygous""","""De novo constitutive""","""EGAN00001352524""","""EGAN00001352525""","""N""","""N""",chr1:1790531,"[""A"",""T""]","""GRCh38"""
"""EGAN00001350451""","""chr1""",1804496,"""T""","""C""","""heterozygous""","""De novo constitutive""","""EGAN00001350453""","""EGAN00001350452""","""N""","""N""",chr1:1804496,"[""T"",""C""]","""GRCh38"""
"""EGAN00001302783""","""chr1""",1804544,"""G""","""A""","""heterozygous""","""De novo constitutive""","""EGAN00001302602""","""EGAN00001302597""","""N""","""N""",chr1:1804544,"[""G"",""A""]","""GRCh38"""


In [14]:
ht=ht.annotate(validated_denovo_inheritance=ht_val_filtered[ht.key].inheritance)

In [15]:
ht.write(f'{lustre_dir}/variant_qc/models/91ba5f38/rf_result_MegaWES_new_with_validated_denovo.ht', overwrite=True)

2021-07-06 10:47:55 Hail: INFO: wrote table with 28803842 rows in 500 partitions to file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/models/91ba5f38/rf_result_MegaWES_new_with_validated_denovo.ht
    Total size: 1.16 GiB
    * Rows: 1.16 GiB
    * Globals: 395.00 B
    * Smallest partition: 57614 rows (2.05 MiB)
    * Largest partition:  57863 rows (2.81 MiB)


In [18]:
run_hash='91ba5f38'
ht = hl.read_table(f'{lustre_dir}/variant_qc/models/{run_hash}_rf_result_FINAL_for_RANKING.ht')

In [19]:
ht=ht.annotate(validated_denovo_inheritance=ht_val_filtered[ht.key].inheritance)

In [21]:
ht.write(f'{lustre_dir}/variant_qc/models/{run_hash}_rf_result_FINAL_for_RANKING_denovo_validated.ht', overwrite=True)

2021-07-06 10:55:20 Hail: INFO: wrote table with 28803842 rows in 500 partitions to file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/models/91ba5f38_rf_result_FINAL_for_RANKING_denovo_validated.ht
    Total size: 2.27 GiB
    * Rows: 2.27 GiB
    * Globals: 395.00 B
    * Smallest partition: 55765 rows (4.16 MiB)
    * Largest partition:  58824 rows (5.78 MiB)


In [3]:
run_hash='91ba5f38'
ht=hl.read_table(f'{lustre_dir}/variant_qc/models/{run_hash}_rf_result_FINAL_for_RANKING_denovo_validated.ht')

In [4]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        n_alt_alleles: int32, 
        MQ: float64, 
        QD: float64, 
        MQRankSum: float64, 
        SOR: float64, 
        ReadPosRankSum: float64
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'MQ': float64 
    'QD': float64 
    'MQRankSum': float64 
    'SOR': float64 
    'ReadPosRankSum': f